In [1]:
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import pandas as pd
import io
import zipfile
import re
print("Libraries imported")

Libraries imported


In [2]:
# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
print("Setting up Chrome options")

PATH = "/Users/sid/Code/chromedriver-mac-arm64/chromedriver"
service = Service(PATH)

print("Initializing WebDriver")
driver = webdriver.Chrome(service=service, options=chrome_options)
csv_files = []

# Function to perform human-like cursor movements
def human_like_hover(driver, element, pause_time=1):
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    time.sleep(pause_time)

print("Opening URL")
driver.get("https://s3.amazonaws.com/tripdata/index.html")

# Wait for the table body to load
try:
    print("Waiting for table body to load")
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "tbody-content"))
    )
    print("Table body loaded successfully")
except Exception as e:
    print(f"Timeout waiting for page to load: {e}")
    driver.quit()
    exit()

# Verify the content of the table body
try:
    print("Finding table body element")
    tbody = driver.find_element(By.ID, "tbody-content")
    print("Table body inner HTML:")
    print(tbody.get_attribute('innerHTML'))
except Exception as e:
    print(f"Error finding elements: {e}")
    driver.quit()
    exit()

# Wait for the links to be present
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, "a"))
    )
    print("Links are present in the table body")
except Exception as e:
    print(f"Timeout waiting for links: {e}")
    driver.quit()
    exit()

# Find all anchor tags within the table body
try:
    print("Finding all anchor tags within the table body")
    links = tbody.find_elements(By.TAG_NAME, "a")
except Exception as e:
    print(f"Error finding elements: {e}")
    driver.quit()
    exit()

print(f"Found {len(links)} links")

# Move cursor over each link and print the href
for link in links:
    try:
        href = link.get_attribute("href")
        if href:
            # Move the cursor to the link
            print(f"Hovering over link: {href}")
            human_like_hover(driver, link, pause_time=random.uniform(0.1, 0.3))
            print(f"Link: {href}")
            if "zip" in href:
                csv_files.append(href)
        # Random delay to mimic human behavior
        time.sleep(random.uniform(0.5, 1))
    except Exception as e:
        print(f"Error processing link: {e}")

print("Closing driver")
driver.quit()

print(f"Found {len(csv_files)} urls files:")

Setting up Chrome options
Initializing WebDriver
Opening URL
Waiting for table body to load
Table body loaded successfully
Finding table body element
Table body inner HTML:

        
Links are present in the table body
Finding all anchor tags within the table body
Found 125 links
Hovering over link: https://s3.amazonaws.com/tripdata/2013-citibike-tripdata.zip
Link: https://s3.amazonaws.com/tripdata/2013-citibike-tripdata.zip
Hovering over link: https://s3.amazonaws.com/tripdata/2014-citibike-tripdata.zip
Link: https://s3.amazonaws.com/tripdata/2014-citibike-tripdata.zip
Hovering over link: https://s3.amazonaws.com/tripdata/2015-citibike-tripdata.zip
Link: https://s3.amazonaws.com/tripdata/2015-citibike-tripdata.zip
Hovering over link: https://s3.amazonaws.com/tripdata/2016-citibike-tripdata.zip
Link: https://s3.amazonaws.com/tripdata/2016-citibike-tripdata.zip
Hovering over link: https://s3.amazonaws.com/tripdata/2017-citibike-tripdata.zip
Link: https://s3.amazonaws.com/tripdata/2017-c

In [3]:
def download_csv(urls):
    """Download the latest version CSV files from nested ZIP archives after filtering out specific directories, and return as a list of pandas DataFrames."""
    all_data = {}
    failed_urls = []

    for url in urls:
        print(f"Downloading {url}...")
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            print(f"Failed to download {url}: {e}")
            failed_urls.append(url)
            continue

        with zipfile.ZipFile(io.BytesIO(response.content)) as the_zip:
            print(f"Found {len(the_zip.namelist())} files in the ZIP archive")
            for file_name in the_zip.namelist():
                if "_MACOSX" in file_name or not file_name.endswith('.csv'):
                    continue  # Skip non-CSV and unwanted system files

                # Ignore empty or placeholder files based on a quick check of the file size if possible
                if the_zip.getinfo(file_name).file_size < 100:
                    continue  # Assumes files smaller than 100 bytes are unlikely to contain useful data

                # Try matching the original format with path
                match = re.match(r".*/(\d{6}-citibike-tripdata)(_?\d*)\.csv", file_name)
                if not match:
                    # Try matching the new format without the path
                    match = re.match(r"(\d{6}-citibike-tripdata)(_?\d*)\.csv", file_name)
                if not match:
                    # Try matching the JC format
                    match = re.match(r"JC-(\d{6}-citibike-tripdata)(_?\d*)\.csv", file_name)

                print(f"Match: {match}")
                if match:
                    print(f"Processing file: {file_name}")
                    base_id = match.group(1)
                    with the_zip.open(file_name) as file:
                        try:
                            # Define dtypes for columns based on the example rows provided
                            dtypes = {
                                'ride_id': 'object',
                                'rideable_type': 'object',
                                'started_at': 'object',
                                'ended_at': 'object',
                                'start_station_name': 'object',
                                'start_station_id': 'float64',
                                'end_station_name': 'object',
                                'end_station_id': 'object',
                                'start_lat': 'float32',
                                'start_lng': 'float32',
                                'end_lat': 'float32',
                                'end_lng': 'float32',
                                'member_casual': 'object',
                                'tripduration': 'float32',
                                'starttime': 'object',
                                'stoptime': 'object',
                                'start_station_id': 'object',
                                'start_station_name': 'object',
                                'start_lat': 'float32',
                                'start_lng': 'float32',
                                'end_station_id': 'object',
                                'end_station_name': 'object',
                                'end_lat': 'float32',
                                'end_lng': 'float32',
                                'bikeid': 'int32',
                                'usertype': 'object',
                                'birth_year': 'float32',
                                'gender': 'object'
                            }
                            temp_df = pd.read_csv(file, dtype=dtypes)
                            if temp_df.shape[0] > 0 and temp_df.shape[1] > 1:
                                if base_id in all_data:
                                    all_data[base_id] = pd.concat([all_data[base_id], temp_df], ignore_index=True)
                                else:
                                    all_data[base_id] = temp_df
                            else:
                                continue  # Skip empty data frames
                        except pd.errors.EmptyDataError:
                            continue  # Skip files with no data

    # Drop duplicates for each DataFrame in all_data
    for key in all_data:
        all_data[key] = all_data[key].drop_duplicates()
    
    datetime_columns = ['started_at', 'ended_at', 'starttime', 'stoptime']
    for key in all_data:
        for col in datetime_columns:
            if col in all_data[key].columns:
                all_data[key][col] = pd.to_datetime(all_data[key][col], errors='coerce')

    # Print failed URLs
    if failed_urls:
        print("Failed to download the following URLs:")
        for url in failed_urls:
            print(url)

    return list(all_data.values()), failed_urls




csv_data_list, failed = download_csv(csv_files)






# Print the shape of each DataFrame to confirm data is loaded




# Print the shape of each DataFrame to confirm data is loaded
print("Data loaded successfully, DF count: ", len(csv_data_list))
print("Failed URLs: ", failed)


Found 40 files in the ZIP archive
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201309-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201309-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201311-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201311-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201307-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201307-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201308-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201308-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201306-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata/201306-citibike-tripdata.csv
Match: <re.Match object; span=(0, 51), match='2013-citibike-tripdata/201310-citibike-tripdata.c>
Processing file: 2013-citibike-tripdata

In [4]:
print("Data loaded successfully, DF count: ", len(csv_data_list))

failed = ['https://s3.amazonaws.com/tripdata/JC-202405-citibike-tripdata.csv.zip']
corrections, failed = download_csv(failed)

Data loaded successfully, DF count:  132
Failed to download https://s3.amazonaws.com/tripdata/JC-202405-citibike-tripdata.csv.zip: 403 Client Error: Forbidden for url: https://s3.amazonaws.com/tripdata/JC-202405-citibike-tripdata.csv.zip
Failed to download the following URLs:
https://s3.amazonaws.com/tripdata/JC-202405-citibike-tripdata.csv.zip


: 

In [5]:
import pandas as pd
from datetime import datetime

def check_month_coverage(dataframes):
    # Expected months for NYC from June 2013 to June 2024
    nyc_start = datetime(2013, 6, 1)
    nyc_end = datetime(2024, 6, 1)
    nyc_months = pd.date_range(nyc_start, nyc_end, freq='MS').strftime('%Y-%m').tolist()

    # Expected months for Jersey City from September 2015 to June 2024, excluding May 2024
    jc_start = datetime(2015, 9, 1)
    jc_end = datetime(2024, 6, 1)
    jc_months = pd.date_range(jc_start, jc_end, freq='MS').strftime('%Y-%m').tolist()
    jc_months.remove('2024-05')

    # Initialize dictionaries to track month coverage
    nyc_coverage = {month: False for month in nyc_months}
    jc_coverage = {month: False for month in jc_months}

    # Define possible columns for station ID and date
    possible_date_cols = ['started_at', 'ended_at', 'starttime', 'stoptime', 'Start Time', 'End Time']
    possible_station_id_cols = ['start_station_id', 'start station id', 'Start Station ID', 'Start_Sation_ID']

    # Check each dataframe
    for df in dataframes:
        # Check for recognized date columns
        date_col = next((col for col in df.columns if col in possible_date_cols), None)
        if date_col:
            df[date_col] = pd.to_datetime(df[date_col])
            df['year_month'] = df[date_col].dt.strftime('%Y-%m')
            year_month = df['year_month'].iloc[0]  # Assuming one month per DataFrame
        else:
            print("No valid date column found. Available columns:", df.columns.tolist())
            continue

        # Check for recognized station ID columns
        station_id_col = next((col for col in df.columns if col in possible_station_id_cols), None)
        if not station_id_col:
            print("No valid station ID column found. Available columns:", df.columns.tolist())
            continue

        # Determine the location based on the sample station ID
        sample_station_id = str(df[station_id_col].iloc[0])
        if 'HB' in sample_station_id or 'JC' in sample_station_id:
            if year_month in jc_months:
                jc_coverage[year_month] = True
            else:
                print(f"Unexpected Jersey City month: {year_month}")
        else:
            if year_month in nyc_months:
                nyc_coverage[year_month] = True
            else:
                print(f"Unexpected NYC month: {year_month}")
                print(f"Sample station ID: {sample_station_id}")

    # Output the results
    missing_nyc = [month for month, covered in nyc_coverage.items() if not covered]
    missing_jc = [month for month, covered in jc_coverage.items() if not covered]

    print("Missing NYC months:", missing_nyc)
    print("Missing Jersey City months:", missing_jc)

# Example usage
check_month_coverage(csv_data_list)
